In [1]:
from lxml import html, etree
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import requests, re, json, time
import pandas as pd

https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"mapBounds":{"west":-81.98262205821744,"east":-81.13804808360807,"south":27.80081193406963,"north":28.85368975780137},"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}},"isListVisible":true}&wants={"cat1":["listResults","mapResults"]}&requestId=9

In [2]:
def default_headers():
    headers = {}
    headers['user-agent'] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"
    headers['accept'] = "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"
    headers['accept-encoding'] = 'gzip, deflate, br'
    headers['accept-language'] = 'en-US,en;q=0.9'
    headers['cache-control'] = 'max-age=0'
    headers['sec-ch-ua'] = '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"'
    headers['sec-ch-ua-mobile'] = '?0'
    headers['sec-ch-ua-platform'] = '"Windows"'
    headers['sec-fetch-dest'] = 'document'
    headers['sec-fetch-mode'] = 'navigate'
    headers['sec-fetch-site'] = 'same-origin'
    headers['sec-fetch-user'] = '?1'
    headers['upgrade-insecure-requests'] = '1'
    return headers

In [40]:
zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"usersSearchTerm":"Orlando, FL","mapBounds":{"west":-82.36371031505338,"east":-80.75695982677213,"south":27.80081193406963,"north":28.85368975780137},"regionSelection":[{"regionId":13121,"regionType":6}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, isListVisible":true,"mapZoom":12}&wants={"cat1":["listResults"]}&requestId=1'
current_page = 1
zillow_results = []

headers = default_headers()
with requests.session() as sess:
    landing = sess.get(zillow_url, headers=headers)
    page = html.fromstring(landing.content)
    zillow_json = json.loads(landing.text)
    total_pages = zillow_json['cat1']['searchList']['totalPages']
    zillow_results.extend(zillow_json['cat1']['searchResults']['listResults'])
    while current_page < total_pages:
        current_page += 1
        print(f"Sleeping for a 5 seconds... getting page {current_page} of {total_pages}")
        time.sleep(5)
        zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{"currentPage":'+str(current_page)+'},"usersSearchTerm":"Orlando, FL","mapBounds":{"west":-82.36371031505338,"east":-80.75695982677213,"south":27.80081193406963,"north":28.85368975780137},"mapZoom":12,"regionSelection":[{"regionId":13121,"regionType":6}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, "isListVisible":true}&wants={"cat1":["listResults"]}&requestId='+str(current_page)
        landing = sess.get(zillow_url, headers=headers)
        page = html.fromstring(landing.content)
        zillow_json = json.loads(landing.text)
        zillow_results.extend(zillow_json['cat1']['searchResults']['listResults'])

In [41]:
df_raw = pd.DataFrame(zillow_results)
df = df_raw.copy()
orlando_df = df.drop(columns=['zpid', 'id','imgSrc', 'hasImage', 'detailUrl', 'statusType', 'countryCurrency', 'unformattedPrice','addressStreet', 'addressCity','addressState', 'addressZipcode', 'isUndisclosedAddress','area', 'latLong', 'isZillowOwned', 'variableData','badgeInfo', 'hdpData', 'isSaved', 'isUserClaimingOwner','isUserConfirmedClaim', 'pgapt', 'sgapt', 'zestimate','shouldShowZestimateAsPrice', 'has3DModel', 'hasVideo', 'isHomeRec','hasAdditionalAttributions', 'isFeaturedListing','list', 'relaxed', 'lotId', 'units', 'buildingName', 'isBuilding','canSaveBuilding'])
orlando_df

,providerListingId,statusText,price,address,beds,baths,availabilityDate
0,11w3zvtcph0be,Apartment for rent,"$1,559/mo","Bella Casa Apartments, 5501 Conroy Rd APT 2, O...",2.0,1.5,2022-08-12 00:00:00
1,3k29nqs8s2g8,Apartment for rent,"$1,799/mo","Bella Vita Apartments, 4700 Cason Cove Dr #2, ...",2.0,2.0,2022-07-15 00:00:00
2,3q4jqbce0u1yw,Apartment for rent,"$1,579+/mo","Conway Forest, 2520 S Conway Rd, Orlando, FL 3...",2.0,1.0,None
3,2ku6mch0x1d7d,Apartment for rent,"$1,669/mo","Coconut Palms, 4618 Middlebrook Rd #2-1, Orlan...",2.0,1.0,2022-06-24 00:00:00
4,19mfzgqe2tuty,Harbor Beach,NaN,"5757 Five Flags Blvd, Orlando, FL",NaN,NaN,NaN
5,59vsreb9qscdz,Apartment for rent,"$1,739/mo","Aria Beach Apartments, 3211 S Semoran Blvd #2,...",2.0,1.5,2022-07-01 00:00:00
6,3uggj8sen7b8u,The Woodlands Apartments,NaN,"604 Laurel Cove Ct, Orlando, FL",NaN,NaN,NaN
7,34kcj0rh3157n,Apartment for rent,"$1,579/mo","Bellagio, 3312 S Semoran Blvd #2, Orlando, FL ...",2.0,2.0,2022-06-10 00:00:00
8,3bfyjjs1z0rk5,Brooke Commons,NaN,"10237 Eastern Lake Ave, Orlando, FL",NaN,NaN,NaN
9,1d8peyqkw4d7p,House for rent,"$1,595/mo","5642 Westview Dr, Orlando, FL 32810",3.0,2.0,2022-06-06 00:00:00


https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"usersSearchTerm":"Kissimmee, FL","mapBounds":{"west":-81.60244099365235,"east":-81.18015400634766,"south":27.95110877554742,"north":28.478113362534792},"regionSelection":[{"regionId":18847,"regionType":6}],"isMapVisible":true,"filterState":{"price":{"min":0,"max":426826},"monthlyPayment":{"min":0,"max":1800},"beds":{"min":2},"baths":{"min":1},"onlyRentalLargeDogsAllowed":{"value":true},"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true}},"isListVisible":true,"mapZoom":11}&wants={"cat1":["mapResults"]}&requestId=2

In [13]:
zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"usersSearchTerm":"Kissimmee, FL","mapBounds":{"west":-81.60244099365235,"east":-81.18015400634766,"south":27.95110877554742,"north":28.478113362534792},"regionSelection":[{"regionId":18847,"regionType":6}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, isListVisible":true,"mapZoom":12}&wants={"cat1":["listResults"]}&requestId=1'
current_page = 1
zillow_results_kissimmee = []

headers = default_headers()
with requests.session() as sess:
    landing = sess.get(zillow_url, headers=headers)
    page = html.fromstring(landing.content)
    zillow_json = json.loads(landing.text)
    total_pages = zillow_json['cat1']['searchList']['totalPages']
    zillow_results_kissimmee.extend(zillow_json['cat1']['searchResults']['listResults'])
    while current_page < total_pages:
        current_page += 1
        print(f"Sleeping for a 5 seconds... getting page {current_page} of {total_pages}")
        time.sleep(5)
        zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{"currentPage":'+str(current_page)+'},"usersSearchTerm":"Kissimmee, FL","mapBounds":{"west":-81.60244099365235,"east":-81.18015400634766,"south":27.95110877554742,"north":28.478113362534792},"mapZoom":12,"regionSelection":[{"regionId":18847,"regionType":6}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, "isListVisible":true}&wants={"cat1":["listResults"]}&requestId='+str(current_page)
        landing = sess.get(zillow_url, headers=headers)
        page = html.fromstring(landing.content)
        zillow_json = json.loads(landing.text)
        zillow_results_kissimmee.extend(zillow_json['cat1']['searchResults']['listResults'])

In [19]:
df_raw = pd.DataFrame(zillow_results_kissimmee)
df = df_raw.copy()
#df
kissimmee_df = df.drop(columns=['zpid', 'id','imgSrc', 'hasImage', 'detailUrl', 'statusType', 'countryCurrency', 'unformattedPrice','addressStreet', 'addressCity','addressState', 'addressZipcode', 'isUndisclosedAddress','area', 'latLong', 'isZillowOwned', 'variableData','badgeInfo', 'hdpData', 'isSaved', 'isUserClaimingOwner','isUserConfirmedClaim', 'pgapt', 'sgapt', 'zestimate','shouldShowZestimateAsPrice', 'has3DModel', 'hasVideo', 'isHomeRec','hasAdditionalAttributions', 'isFeaturedListing','list', 'relaxed'])
kissimmee_df

,providerListingId,statusText,price,address,beds,baths,availabilityDate
0,1mxg2yupkf71q,Apartment for rent,"$1,745/mo","1945 Grand Oak Dr, Kissimmee, FL 34744",2,2.0,2022-05-17 00:00:00
1,qk01fs7d8g3g,Apartment for rent,"$1,645/mo","1955 Grand Oak Dr, Kissimmee, FL 34744",2,2.0,2022-05-17 00:00:00
2,4e6cjyhrcvb3v,House for rent,"$1,800/mo","4401 White Oak Cir, Kissimmee, FL 34746",3,2.0,2022-05-02 00:00:00


https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"usersSearchTerm":"Poinciana, FL","mapBounds":{"west":-81.56897824682618,"east":-81.35783475317383,"south":28.01330703289227,"north":28.276981638025106},"regionSelection":[{"regionId":40387,"regionType":6}],"isMapVisible":true,"filterState":{"price":{"min":0,"max":426826},"monthlyPayment":{"min":0,"max":1800},"beds":{"min":2},"baths":{"min":1},"onlyRentalLargeDogsAllowed":{"value":true},"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true}},"isListVisible":true,"mapZoom":12}&wants={"cat1":["mapResults"]}&requestId=2

In [20]:
zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"usersSearchTerm":"Poinciana, FL","mapBounds":{"west":-81.56897824682618,"east":-81.35783475317383,"south":28.01330703289227,"north":28.276981638025106},"regionSelection":[{"regionId":40387,"regionType":6}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, isListVisible":true,"mapZoom":12}&wants={"cat1":["listResults"]}&requestId=1'
current_page = 1
zillow_results_poinciana = []

headers = default_headers()
with requests.session() as sess:
    landing = sess.get(zillow_url, headers=headers)
    page = html.fromstring(landing.content)
    zillow_json = json.loads(landing.text)
    total_pages = zillow_json['cat1']['searchList']['totalPages']
    zillow_results_poinciana.extend(zillow_json['cat1']['searchResults']['listResults'])
    while current_page < total_pages:
        current_page += 1
        print(f"Sleeping for a 5 seconds... getting page {current_page} of {total_pages}")
        time.sleep(5)
        zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{"currentPage":'+str(current_page)+'},"usersSearchTerm":"Kissimmee, FL","mapBounds":{"west":-81.56897824682618,"east":-81.35783475317383,"south":28.01330703289227,"north":28.276981638025106},"mapZoom":12,"regionSelection":[{"regionId":40387,"regionType":6}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, "isListVisible":true}&wants={"cat1":["listResults"]}&requestId='+str(current_page)
        landing = sess.get(zillow_url, headers=headers)
        page = html.fromstring(landing.content)
        zillow_json = json.loads(landing.text)
        zillow_results_poinciana.extend(zillow_json['cat1']['searchResults']['listResults'])

In [21]:
df_raw = pd.DataFrame(zillow_results_poinciana)
df = df_raw.copy()
#df
poinciana_df = df.drop(columns=['zpid', 'id','imgSrc', 'hasImage', 'detailUrl', 'statusType', 'countryCurrency', 'unformattedPrice','addressStreet', 'addressCity','addressState', 'addressZipcode', 'isUndisclosedAddress','area', 'latLong', 'isZillowOwned', 'variableData','badgeInfo', 'hdpData', 'isSaved', 'isUserClaimingOwner','isUserConfirmedClaim', 'pgapt', 'sgapt', 'zestimate','shouldShowZestimateAsPrice', 'has3DModel', 'hasVideo', 'isHomeRec','hasAdditionalAttributions', 'isFeaturedListing','list', 'relaxed'])
poinciana_df

,providerListingId,statusText,price,address,beds,baths,availabilityDate
0,31ka7keq5efca,House for rent,"$1,695/mo","833 Blanc Ct, Kissimmee, FL 34759",3,2.0,None
1,38v1zvcy0neu4,House for rent,"$1,795/mo","1204 Atlantic Ct, Poinciana, FL 34759",4,2.0,None
2,vxnt1x6tkqjn,House for rent,"$1,775/mo","610 Heraldo Ct, Kissimmee, FL 34758",3,2.0,None
3,43qfjfdqdpjb3,House for rent,"$1,600/mo","321 Caldbeck Way, Kissimmee, FL 34758",3,2.0,2022-06-10 00:00:00


https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"usersSearchTerm":"Winter Park, FL","mapBounds":{"west":-81.43352674682617,"east":-81.22238325317383,"south":28.473762139132905,"north":28.736296072609846},"regionSelection":[{"regionId":34884,"regionType":6}],"isMapVisible":true,"filterState":{"price":{"min":0,"max":426826},"monthlyPayment":{"min":0,"max":1800},"beds":{"min":2},"baths":{"min":1},"onlyRentalLargeDogsAllowed":{"value":true},"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true}},"isListVisible":true,"mapZoom":12}&wants={"cat1":["mapResults"]}&requestId=2

In [25]:
zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"usersSearchTerm":"Winter Park, FL","mapBounds":{"west":-81.43352674682617,"east":-81.22238325317383,"south":28.473762139132905,"north":28.736296072609846},"regionSelection":[{"regionId":34884,"regionType":6}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, isListVisible":true,"mapZoom":12}&wants={"cat1":["listResults"]}&requestId=1'
current_page = 1
zillow_results_WinterPark = []

headers = default_headers()
with requests.session() as sess:
    landing = sess.get(zillow_url, headers=headers)
    page = html.fromstring(landing.content)
    zillow_json = json.loads(landing.text)
    total_pages = zillow_json['cat1']['searchList']['totalPages']
    zillow_results_WinterPark.extend(zillow_json['cat1']['searchResults']['listResults'])
    while current_page < total_pages:
        current_page += 1
        print(f"Sleeping for a 5 seconds... getting page {current_page} of {total_pages}")
        time.sleep(5)
        zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{"currentPage":'+str(current_page)+'},"usersSearchTerm":"Winter Park, FL","mapBounds":{"west":-81.43352674682617,"east":-81.22238325317383,"south":28.473762139132905,"north":28.736296072609846},"mapZoom":12,"regionSelection":[{"regionId":34884,"regionType":6}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, "isListVisible":true}&wants={"cat1":["listResults"]}&requestId='+str(current_page)
        landing = sess.get(zillow_url, headers=headers)
        page = html.fromstring(landing.content)
        zillow_json = json.loads(landing.text)
        zillow_results_WinterPark.extend(zillow_json['cat1']['searchResults']['listResults'])

In [26]:
df_raw = pd.DataFrame(zillow_results_WinterPark)
df = df_raw.copy()
#df
WinterPark_df = df.drop(columns=['zpid', 'id','imgSrc', 'hasImage', 'detailUrl', 'statusType', 'countryCurrency', 'unformattedPrice','addressStreet', 'addressCity','addressState', 'addressZipcode', 'isUndisclosedAddress','area', 'latLong', 'isZillowOwned', 'variableData','badgeInfo', 'hdpData', 'isSaved', 'isUserClaimingOwner','isUserConfirmedClaim', 'pgapt', 'sgapt', 'zestimate','shouldShowZestimateAsPrice', 'has3DModel', 'hasVideo', 'isHomeRec','hasAdditionalAttributions', 'isFeaturedListing','list', 'relaxed','streetViewMetadataURL','streetViewURL'])
WinterPark_df

,providerListingId,statusText,price,address,beds,baths,availabilityDate
0,53gw0519vb2rp,Apartment for rent,"$1,789/mo","Sapphire Winter Park Apartments, 1743 Semoran ...",2,2.0,2022-06-10 00:00:00
1,32bq93sktfpvm,Apartment for rent,"$1,719/mo","Azure Winter Park, 3650 Summer Wind Dr #2-P, W...",2,2.0,2022-06-01 00:00:00
2,vxw33nyr9h7x,House for rent,"$1,200/mo","100 Stefanik Rd, Winter Park, FL 32792",2,1.5,None


https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"usersSearchTerm":"Haines City, FL","mapBounds":{"west":-81.77262249365235,"east":-81.35033550634766,"south":27.824796518413915,"north":28.352421645405734},"regionSelection":[{"regionId":45612,"regionType":6}],"isMapVisible":true,"filterState":{"price":{"min":0,"max":426826},"monthlyPayment":{"min":0,"max":1800},"beds":{"min":2},"baths":{"min":1},"onlyRentalLargeDogsAllowed":{"value":true},"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true}},"isListVisible":true,"mapZoom":11}&wants={"cat1":["mapResults"]}&requestId=2

In [27]:
zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"usersSearchTerm":"Haines City, FL","mapBounds":{"west":-81.77262249365235,"east":-81.35033550634766,"south":27.824796518413915,"north":28.352421645405734},"regionSelection":[{"regionId":45612,"regionType":6}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, isListVisible":true,"mapZoom":12}&wants={"cat1":["listResults"]}&requestId=1'
current_page = 1
zillow_results_HainesCity = []

headers = default_headers()
with requests.session() as sess:
    landing = sess.get(zillow_url, headers=headers)
    page = html.fromstring(landing.content)
    zillow_json = json.loads(landing.text)
    total_pages = zillow_json['cat1']['searchList']['totalPages']
    zillow_results_HainesCity.extend(zillow_json['cat1']['searchResults']['listResults'])
    while current_page < total_pages:
        current_page += 1
        print(f"Sleeping for a 5 seconds... getting page {current_page} of {total_pages}")
        time.sleep(5)
        zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{"currentPage":'+str(current_page)+'},"usersSearchTerm":"Haines City, FL","mapBounds":{"west":-81.77262249365235,"east":-81.35033550634766,"south":27.824796518413915,"north":28.352421645405734},"mapZoom":12,"regionSelection":[{"regionId":45612,"regionType":6}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, "isListVisible":true}&wants={"cat1":["listResults"]}&requestId='+str(current_page)
        landing = sess.get(zillow_url, headers=headers)
        page = html.fromstring(landing.content)
        zillow_json = json.loads(landing.text)
        zillow_results_HainesCity.extend(zillow_json['cat1']['searchResults']['listResults'])

In [29]:
df_raw = pd.DataFrame(zillow_results_HainesCity)
df = df_raw.copy()
#df
HainesCity_df = df.drop(columns=['zpid', 'id','imgSrc', 'hasImage', 'detailUrl', 'statusType', 'countryCurrency', 'unformattedPrice','addressStreet', 'addressCity','addressState', 'addressZipcode', 'isUndisclosedAddress','area', 'latLong', 'isZillowOwned', 'variableData','badgeInfo', 'hdpData', 'isSaved', 'isUserClaimingOwner','isUserConfirmedClaim', 'pgapt', 'sgapt', 'zestimate','shouldShowZestimateAsPrice', 'has3DModel', 'hasVideo', 'isHomeRec','hasAdditionalAttributions', 'isFeaturedListing','list', 'relaxed'])
HainesCity_df

,providerListingId,statusText,price,address,beds,baths,availabilityDate
0,441w85sy4dv24,House for rent,"$1,775/mo","2611 Crest Dr, Haines City, FL 33844",3,2.0,2022-06-30 00:00:00


https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"usersSearchTerm":"Polk County, FL","mapBounds":{"west":-82.46321397460937,"east":-80.77406602539062,"south":26.94192490182362,"north":29.054028972257914},"regionSelection":[{"regionId":285,"regionType":4}],"isMapVisible":true,"filterState":{"price":{"min":0,"max":426826},"monthlyPayment":{"min":0,"max":1800},"beds":{"min":2},"baths":{"min":1},"onlyRentalLargeDogsAllowed":{"value":true},"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true}},"isListVisible":true,"mapZoom":9}&wants={"cat1":["mapResults"]}&requestId=2

In [30]:
zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"usersSearchTerm":"Polk County, FL","mapBounds":{"west":-82.46321397460937,"east":-80.77406602539062,"south":26.94192490182362,"north":29.054028972257914},"regionSelection":[{"regionId":285,"regionType":4}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, isListVisible":true,"mapZoom":12}&wants={"cat1":["listResults"]}&requestId=1'
current_page = 1
zillow_results_PolkCounty = []

headers = default_headers()
with requests.session() as sess:
    landing = sess.get(zillow_url, headers=headers)
    page = html.fromstring(landing.content)
    zillow_json = json.loads(landing.text)
    total_pages = zillow_json['cat1']['searchList']['totalPages']
    zillow_results_PolkCounty.extend(zillow_json['cat1']['searchResults']['listResults'])
    while current_page < total_pages:
        current_page += 1
        print(f"Sleeping for a 5 seconds... getting page {current_page} of {total_pages}")
        time.sleep(5)
        zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{"currentPage":'+str(current_page)+'},"usersSearchTerm":"Polk County, FL","mapBounds":{"west":-82.46321397460937,"east":-80.77406602539062,"south":26.94192490182362,"north":29.054028972257914},"mapZoom":12,"regionSelection":[{"regionId":285,"regionType":4}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, "isListVisible":true}&wants={"cat1":["listResults"]}&requestId='+str(current_page)
        landing = sess.get(zillow_url, headers=headers)
        page = html.fromstring(landing.content)
        zillow_json = json.loads(landing.text)
        zillow_results_PolkCounty.extend(zillow_json['cat1']['searchResults']['listResults'])

Sleeping for a 5 seconds... getting page 2 of 2


In [ ]:
df_raw = pd.DataFrame(zillow_results_PolkCounty)
df = df_raw.copy()
#df
PolkCounty_df = df.drop(columns=['zpid', 'id','imgSrc', 'hasImage', 'detailUrl', 'statusType', 'countryCurrency', 'unformattedPrice','addressStreet', 'addressCity','addressState', 'addressZipcode', 'isUndisclosedAddress','area', 'latLong', 'isZillowOwned', 'variableData','badgeInfo', 'hdpData', 'isSaved', 'isUserClaimingOwner','isUserConfirmedClaim', 'pgapt', 'sgapt', 'zestimate','shouldShowZestimateAsPrice', 'has3DModel', 'hasVideo', 'isHomeRec','hasAdditionalAttributions', 'isFeaturedListing','list', 'relaxed'])
PolkCounty_df

https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"usersSearchTerm":"Orange County, FL","mapBounds":{"west":-81.68304048730468,"east":-80.8384665126953,"south":28.040070283961438,"north":29.090580631522116},"regionSelection":[{"regionId":1287,"regionType":4}],"isMapVisible":true,"filterState":{"price":{"min":0,"max":426826},"monthlyPayment":{"min":0,"max":1800},"beds":{"min":2},"baths":{"min":1},"onlyRentalLargeDogsAllowed":{"value":true},"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true}},"isListVisible":true}&wants={"cat1":["mapResults"]}&requestId=2

In [32]:
zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"usersSearchTerm":"Orange County, FL","mapBounds":{"west":-81.68304048730468,"east":-80.8384665126953,"south":28.040070283961438,"north":29.090580631522116},"regionSelection":[{"regionId":1287,"regionType":4}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, isListVisible":true,"mapZoom":12}&wants={"cat1":["listResults"]}&requestId=1'
current_page = 1
zillow_results_OrangeCounty = []

headers = default_headers()
with requests.session() as sess:
    landing = sess.get(zillow_url, headers=headers)
    page = html.fromstring(landing.content)
    zillow_json = json.loads(landing.text)
    total_pages = zillow_json['cat1']['searchList']['totalPages']
    zillow_results_OrangeCounty.extend(zillow_json['cat1']['searchResults']['listResults'])
    while current_page < total_pages:
        current_page += 1
        print(f"Sleeping for a 5 seconds... getting page {current_page} of {total_pages}")
        time.sleep(5)
        zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{"currentPage":'+str(current_page)+'},"usersSearchTerm":"Orange County, FL","mapBounds":{"west":-81.68304048730468,"east":-80.8384665126953,"south":28.040070283961438,"north":29.090580631522116},"mapZoom":12,"regionSelection":[{"regionId":1287,"regionType":4}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, "isListVisible":true}&wants={"cat1":["listResults"]}&requestId='+str(current_page)
        landing = sess.get(zillow_url, headers=headers)
        page = html.fromstring(landing.content)
        zillow_json = json.loads(landing.text)
        zillow_results_OrangeCounty.extend(zillow_json['cat1']['searchResults']['listResults'])

Sleeping for a 5 seconds... getting page 2 of 2


In [ ]:
df_raw = pd.DataFrame(zillow_results_OrangeCounty)
df = df_raw.copy()
#df
OrangeCounty_df = df.drop(columns=['zpid', 'id','imgSrc', 'hasImage', 'detailUrl', 'statusType', 'lotId', 'units', 'buildingName', 'isBuilding', 'canSaveBuilding', 'countryCurrency', 'unformattedPrice','addressStreet', 'addressCity','addressState', 'addressZipcode', 'isUndisclosedAddress','area', 'latLong', 'isZillowOwned', 'variableData','badgeInfo', 'hdpData', 'isSaved', 'isUserClaimingOwner','isUserConfirmedClaim', 'pgapt', 'sgapt', 'zestimate','shouldShowZestimateAsPrice', 'has3DModel', 'hasVideo', 'isHomeRec','hasAdditionalAttributions', 'isFeaturedListing','list', 'relaxed'])
OrangeCounty_df

https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"usersSearchTerm":"Seminole County, FL","mapBounds":{"west":-81.6455709873047,"east":-80.80099701269532,"south":28.21915925369373,"north":29.26788569260213},"regionSelection":[{"regionId":2538,"regionType":4}],"isMapVisible":true,"filterState":{"price":{"min":0,"max":426826},"monthlyPayment":{"min":0,"max":1800},"beds":{"min":2},"baths":{"min":1},"onlyRentalLargeDogsAllowed":{"value":true},"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true}},"isListVisible":true}&wants={"cat1":["mapResults"]}&requestId=2

In [34]:
zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{},"usersSearchTerm":"Seminole County, FL","mapBounds":{"west":-81.6455709873047,"east":-80.80099701269532,"south":28.21915925369373,"north":29.26788569260213},"regionSelection":[{"regionId":2538,"regionType":4}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, isListVisible":true,"mapZoom":12}&wants={"cat1":["listResults"]}&requestId=1'
current_page = 1
zillow_results_SeminoleCounty = []

headers = default_headers()
with requests.session() as sess:
    landing = sess.get(zillow_url, headers=headers)
    page = html.fromstring(landing.content)
    zillow_json = json.loads(landing.text)
    total_pages = zillow_json['cat1']['searchList']['totalPages']
    zillow_results_SeminoleCounty.extend(zillow_json['cat1']['searchResults']['listResults'])
    while current_page < total_pages:
        current_page += 1
        print(f"Sleeping for a 5 seconds... getting page {current_page} of {total_pages}")
        time.sleep(5)
        zillow_url = 'https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState={"pagination":{"currentPage":'+str(current_page)+'},"usersSearchTerm":"Seminole County, FL","mapBounds":{"west":-81.6455709873047,"east":-80.80099701269532,"south":28.21915925369373,"north":29.26788569260213},"mapZoom":12,"regionSelection":[{"regionId":2538,"regionType":4}],"isMapVisible":true,"filterState":{"isForSaleByAgent":{"value":false},"isForSaleByOwner":{"value":false},"isNewConstruction":{"value":false},"isForSaleForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isForRent":{"value":true},"isAllHomes":{"value":true},"beds":{"min":2},"baths":{"min":1},"monthlyPayment":{"max":1800},"price":{"max":426826},"onlyRentalLargeDogsAllowed":{"value":true}}, "isListVisible":true}&wants={"cat1":["listResults"]}&requestId='+str(current_page)
        landing = sess.get(zillow_url, headers=headers)
        page = html.fromstring(landing.content)
        zillow_json = json.loads(landing.text)
        zillow_results_SeminoleCounty.extend(zillow_json['cat1']['searchResults']['listResults'])

In [38]:
df_raw = pd.DataFrame(zillow_results_SeminoleCounty)
df = df_raw.copy()
#df
SeminoleCounty_df = df.drop(columns=['zpid', 'id','imgSrc', 'hasImage', 'detailUrl', 'statusType', 'lotId', 'units', 'buildingName', 'isBuilding', 'canSaveBuilding', 'countryCurrency', 'unformattedPrice','addressStreet', 'addressCity','addressState', 'addressZipcode', 'isUndisclosedAddress','area', 'latLong', 'isZillowOwned', 'variableData','badgeInfo', 'hdpData', 'isSaved', 'isUserClaimingOwner','isUserConfirmedClaim', 'pgapt', 'sgapt', 'zestimate','shouldShowZestimateAsPrice', 'has3DModel', 'hasVideo', 'isHomeRec','hasAdditionalAttributions', 'isFeaturedListing','list', 'relaxed','streetViewMetadataURL', 'streetViewURL'])
SeminoleCounty_df

,providerListingId,statusText,price,address,beds,baths,availabilityDate
0,53gw0519vb2rp,Apartment for rent,"$1,789/mo","Sapphire Winter Park Apartments, 1743 Semoran ...",2.0,2.0,2022-06-10 00:00:00
1,375b3ayqb5w9a,House for rent,"$1,725/mo","705 W 24th Pl, Sanford, FL 32771",3.0,2.0,None
2,1zecy6jt2bey8,House for rent,"$1,795/mo","109 Placid Woods Ct, Sanford, FL 32773",3.0,2.0,None
3,5gu108jkqs4kv,House for rent,"$1,745/mo","2191 Granby St, Sanford, FL 32771",3.0,1.0,2022-06-17 00:00:00
4,2m4dhntzu7u10,House for rent,$700/mo,"2340 Broadway St, Sanford, FL 32771",3.0,1.0,2022-06-01 00:00:00
5,3zugg75md8b0x,Townhouse for rent,"$1,700/mo","(undisclosed Address), Altamonte Springs, FL 3...",2.0,2.0,2022-05-23 00:00:00
6,vxw33nyr9h7x,House for rent,"$1,200/mo","100 Stefanik Rd, Winter Park, FL 32792",2.0,1.5,None
7,3uzd52mp3jk5u,Apartment for rent,"$1,550/mo","51 Saint Johanna Dr #51, Oviedo, FL 32765",2.0,2.0,None
8,39dr8hn3zuc4h,House for rent,"$1,400/mo","1211 Marie Ave, Apopka, FL 32703",3.0,2.0,2022-06-01 00:00:00
9,5hdkjg675wdud,House for rent,"$1,650/mo","1074 Timberlane Trl, Casselberry, FL 32707",3.0,2.0,None


In [42]:
concatenated = pd.concat([orlando_df,kissimmee_df,poinciana_df,WinterPark_df,HainesCity_df,PolkCounty_df,OrangeCounty_df,SeminoleCounty_df])
concatenated

,providerListingId,statusText,price,address,beds,baths,availabilityDate,lotId,units,buildingName,isBuilding,canSaveBuilding
0,11w3zvtcph0be,Apartment for rent,"$1,559/mo","Bella Casa Apartments, 5501 Conroy Rd APT 2, O...",2.0,1.5,2022-08-12 00:00:00,NaN,NaN,NaN,NaN,NaN
1,3k29nqs8s2g8,Apartment for rent,"$1,799/mo","Bella Vita Apartments, 4700 Cason Cove Dr #2, ...",2.0,2.0,2022-07-15 00:00:00,NaN,NaN,NaN,NaN,NaN
2,3q4jqbce0u1yw,Apartment for rent,"$1,579+/mo","Conway Forest, 2520 S Conway Rd, Orlando, FL 3...",2.0,1.0,None,NaN,NaN,NaN,NaN,NaN
3,2ku6mch0x1d7d,Apartment for rent,"$1,669/mo","Coconut Palms, 4618 Middlebrook Rd #2-1, Orlan...",2.0,1.0,2022-06-24 00:00:00,NaN,NaN,NaN,NaN,NaN
4,19mfzgqe2tuty,Harbor Beach,NaN,"5757 Five Flags Blvd, Orlando, FL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6,vxw33nyr9h7x,House for rent,"$1,200/mo","100 Stefanik Rd, Winter Park, FL 32792",2.0,1.5,None,NaN,NaN,NaN,NaN,NaN
7,3uzd52mp3jk5u,Apartment for rent,"$1,550/mo","51 Saint Johanna Dr #51, Oviedo, FL 32765",2.0,2.0,None,NaN,NaN,NaN,NaN,NaN
8,39dr8hn3zuc4h,House for rent,"$1,400/mo","1211 Marie Ave, Apopka, FL 32703",3.0,2.0,2022-06-01 00:00:00,NaN,NaN,NaN,NaN,NaN
9,5hdkjg675wdud,House for rent,"$1,650/mo","1074 Timberlane Trl, Casselberry, FL 32707",3.0,2.0,None,NaN,NaN,NaN,NaN,NaN


In [44]:
master_list = concatenated.drop(columns=['lotId', 'units','buildingName','isBuilding','canSaveBuilding'])
master_list

,providerListingId,statusText,price,address,beds,baths,availabilityDate
0,11w3zvtcph0be,Apartment for rent,"$1,559/mo","Bella Casa Apartments, 5501 Conroy Rd APT 2, O...",2.0,1.5,2022-08-12 00:00:00
1,3k29nqs8s2g8,Apartment for rent,"$1,799/mo","Bella Vita Apartments, 4700 Cason Cove Dr #2, ...",2.0,2.0,2022-07-15 00:00:00
2,3q4jqbce0u1yw,Apartment for rent,"$1,579+/mo","Conway Forest, 2520 S Conway Rd, Orlando, FL 3...",2.0,1.0,None
3,2ku6mch0x1d7d,Apartment for rent,"$1,669/mo","Coconut Palms, 4618 Middlebrook Rd #2-1, Orlan...",2.0,1.0,2022-06-24 00:00:00
4,19mfzgqe2tuty,Harbor Beach,NaN,"5757 Five Flags Blvd, Orlando, FL",NaN,NaN,NaN
...,...,...,...,...,...,...,...
6,vxw33nyr9h7x,House for rent,"$1,200/mo","100 Stefanik Rd, Winter Park, FL 32792",2.0,1.5,None
7,3uzd52mp3jk5u,Apartment for rent,"$1,550/mo","51 Saint Johanna Dr #51, Oviedo, FL 32765",2.0,2.0,None
8,39dr8hn3zuc4h,House for rent,"$1,400/mo","1211 Marie Ave, Apopka, FL 32703",3.0,2.0,2022-06-01 00:00:00
9,5hdkjg675wdud,House for rent,"$1,650/mo","1074 Timberlane Trl, Casselberry, FL 32707",3.0,2.0,None


In [46]:
master_list.drop_duplicates()

,providerListingId,statusText,price,address,beds,baths,availabilityDate
0,11w3zvtcph0be,Apartment for rent,"$1,559/mo","Bella Casa Apartments, 5501 Conroy Rd APT 2, O...",2.0,1.5,2022-08-12 00:00:00
1,3k29nqs8s2g8,Apartment for rent,"$1,799/mo","Bella Vita Apartments, 4700 Cason Cove Dr #2, ...",2.0,2.0,2022-07-15 00:00:00
2,3q4jqbce0u1yw,Apartment for rent,"$1,579+/mo","Conway Forest, 2520 S Conway Rd, Orlando, FL 3...",2.0,1.0,None
3,2ku6mch0x1d7d,Apartment for rent,"$1,669/mo","Coconut Palms, 4618 Middlebrook Rd #2-1, Orlan...",2.0,1.0,2022-06-24 00:00:00
4,19mfzgqe2tuty,Harbor Beach,NaN,"5757 Five Flags Blvd, Orlando, FL",NaN,NaN,NaN
...,...,...,...,...,...,...,...
5,3zugg75md8b0x,Townhouse for rent,"$1,700/mo","(undisclosed Address), Altamonte Springs, FL 3...",2.0,2.0,2022-05-23 00:00:00
7,3uzd52mp3jk5u,Apartment for rent,"$1,550/mo","51 Saint Johanna Dr #51, Oviedo, FL 32765",2.0,2.0,None
8,39dr8hn3zuc4h,House for rent,"$1,400/mo","1211 Marie Ave, Apopka, FL 32703",3.0,2.0,2022-06-01 00:00:00
9,5hdkjg675wdud,House for rent,"$1,650/mo","1074 Timberlane Trl, Casselberry, FL 32707",3.0,2.0,None


In [47]:
master_list.to_csv('Houses_May_30.csv')